### Demo Notebook For CPO 2.0 API
This notebook presents how to connect to the CPO 2.0 API, manage files, set up parameters and request predictions. It also covers the requirement on and format of input data.

In [1]:
// Load the binary
#r "QuantConnect.PredictNowNET.dll"

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
// Import statement
using System;
using System.IO;
using QuantConnect.Configuration;
using QuantConnect.PredictNowNET;
using QuantConnect.PredictNowNET.Models;

In [3]:
Config.Set("predict-now-url", "http://localhost:5005/");
var userEmail = "me@office.com";
var client = new PredictNowClient(userEmail);
client.Connected

True

### Uploading Data
CPO requires daily returns data and constraints on each portfolio components as inputs. User can also upload additional features. These data are stored in separate directories under the client's account. If a new data file is re-uploaded under the same file name, the existing file will be overwritten. This can be used to update the return file for live predictions.

#### Return File
Return file should contain daily return of all or a superset of the components in the portfolio universe, with `return(t) = close(t) / close(t-1) - 1.0`. The return file should have a `Date` (alias `tradedate`, `time`, `timestamp`, `datetime`) column for indexing purposes, and the desired format is `yyyy-mm-dd`. Upon uploading the return file, CPO API will identify the `Date` column and provides feedbacks on the start- and end-dates so user can check if the processing is successful. User can upload multiple return files but only one return file can be used for each portfolio.

CPO API will take all other columns as daily returns of portfolio components. NaNs and zeros and treated differently for these return columns. To be specific, NaNs are for historical period when the ticker *does not exist*, while zeros are taken as *real zero returns* (e.g. due to holidays). *Please contact PredictNow if your return data is sparse.*

The following code upload the `ETF_return.csv` file from the `Data` directory, then list all available return files under the account.

In [4]:
var filename = Path.Combine("..", "..", "..", "Tests", "Data", "ETF_return.csv");
var message = client.UploadReturnsFile(filename);
message

Date info from date with 16 return columns. Index range between 2010-01-04 and 2023-03-03.

In [5]:
// List the files
var files = client.ListReturnsFiles();
files

[ ETF_return.csv, ETF_return2.csv, ETF_return99.csv, CUsersAlexPredictNowNETTestsbinDebugnet6.0DataETF_return.csv, Returns.csv ]

#### Constrain files
The constraint file defines the portfolio universe and the upper and lower boundaries (min and max allocation) of each components. There should be a `component` column for the component names, and these names should also be a subset of the return columns of the return file to be used. The column names are case sensitive. The upper and lower boundaries of each component should be given in the `UB` and `LB` columns, and their default values are 0.0 (0%) and 1.0 (100%) if not provided for a given component. Each user can upload multiple constraint files under different names, but only one constraint file can be used for each portfoio.

The following code upload the `ETF_constrain.csv` file from the `Data` directory, then list all available constraint files under the account. Note that `ETF_constrain.csv` contains fewer components than the `ETF_return.csv` file, in which case the additional ETFs will not be used to construct the portfolio.

In [6]:
var filename = Path.Combine("..", "..", "..", "Tests", "Data", "ETF_constrain.csv");
var message = client.UploadConstraintFile(filename);
message

Constraint file processed for 6 components

In [7]:
// List the files
var files = client.ListConstraintFiles();
files

[ ETF_constrain.csv, Constraint.csv ]

#### Feature files (Optional)

User can provide additional feature files to help prediction. Feature files are time-series files just like returns, so the key column is also `Date` (alias `tradedate`, `time`, `timestamp`, `datetime`). The desired format is `yyyy-mm-dd` for the ease of auto-processing. The clinet's features are merged with pre-engineered features at PredictNow by matching the `Date` column, and CPO API assumes all features are available before the open of market of the target date. All features are forward filled (using the latest one that is available) before training. User can upload multiple feature files.

The following code upload the `Random_Feature.csv` file from the `Data` directory, then list all available client feauture files under the account. The uploaded feature are randomly generated and should not provide any prediction power.

In [8]:
var filename = Path.Combine("..", "..", "..", "Tests", "Data", "Random_Feature.csv");
var message = client.UploadFeaturesFile(filename);
message

Date info from TradeDate with 4 return columns. Index range between 2015-01-02 and 2022-08-02.

In [9]:
// List the files
var files = client.ListFeaturesFiles();
files

[ Random_Feature.csv, features.csv ]

### Parameters

There are certain parameters that are required to run a CPO job, and these parameters generally fall into two categoraries: a) those define the property of the target portfolio, like what components are included, how often it is rebalanced, what is the metric to rebalanced etc., and b) parameter that define the job, like what period of time should be used as in-sample, or what is the target date for a live prediction. 

This section focuses on the first part of the parameters, while the job-related parameters will be described in later sections. But before diving into the parameters, we will talke about the important idea of *project* in CPO API.

#### Project

CPO is designed to determine the allocation of a portfolio during each rebalancing, but not for ticker selection or tune the optimized rebalancing frequency. These configurations are like hyper-parameters of the portfolio, and users can use projects to test different portfolio configurations like:

- What if I do weekly rebalancing instead of monthly rebalancing?
- What if I lower the max allocation of some high-risk components and allow certain amount of Cash allocation?
- What if I include additional features that I think might be helpful?
etc.

In short, each project corresponds to a specific portfolio set up that is defined by the portfolio parameters.

#### Portfolio parameters
The following parameters define the portfolio and should be fixed for a given project throughthout in-sample, out-of-sample, and live-prediction. 

- `name`, mandatory, project / portfolio identification, e.g. `Demo_Project`
- `returnsFile`, mandatory, daily return filename, should be uploaded first, e.g. `ETF_return.csv`
- `constraintFile`, mandatory, portfolio components and their min and max allocations, should be uploaded first, e.g. `ETF_constraint.csv`. Note it's the `component` column in the `constraint_file` that defines the current portfolio universe.
- `maxCash`, mandatory, maximum cash allocation allowed when risk is predicted to be large, float between 0 and 1 where 1 correspond to 100% (no market exposure).
- `rebalancingPeriodUnit`, mandatory, 'week' or 'month', used with `rebalancingPeriod`.
- `rebalancingPeriod`, mandatory, int. If `rebalancingPeriod = 2` and `rebalancingPeriodUnit = 'week'`, the portfolio would rebalanced every other week.
- `rebalanceOn`, mandatory, 'first' or 'last', determine if the portfolio is rebalanced at the close of the first or last market day of the rebalancing period. Please see next subsection on how the dates are indexed in CPO.
- `trainingDataSize`, mandatory, int, in the unit of years, size of rolling window that is used to make the prediction for each rebalancing.
- `evaluationMetric`, mandatory, key performance metric to optimize, can be chosen from 'return', 'risk', 'sharpe', 'CAGR', 'UI', 'UPI', or 'MaxDD'. For risk related metrics, i.e. 'risk', 'UI', and 'MaxDD', `max_cash` will be overridden since Cash will always have zero and hence the minimal risk.
- `featureFile`, optional, 'none' or 'feature_file1.csv, feature_file2.csv, ...', clinet feature to be included, and these files should be uploaded first.
- `skipPNowFeature`, optional, if set to 'yes', 'true', or 'skip', will not include predictnow features. Note, if `feature_file` is not provided (`none` or not in the parameter dictionary) there would be no X-columns in the prediction model.

The code below gives how to set up the portfolio parameters, with some optional ones commented out.

In [10]:
var portfolioParameters = new PortfolioParameters(
    name: "Demo_Project_20231211",
    returnsFile: "ETF_return.csv",
    constraintFile: "ETF_constrain.csv",
    maxCash: 1.0,
    rebalancingPeriodUnit: "month",
    rebalancingPeriod: 1,
    rebalanceOn: "first",
    trainingDataSize: 3,
    evaluationMetric: "sharpe");

#### Additional Notes on how CPO Handles Date index
There are two sets of date that are tightly related to each other, the rebalance period, and the rebalancing date.

Rebalance period is usually labelled by the first calendar day within the period, regardless of if it is a valid market day. The start date of the first rebalancing period is taken from the `trainingStartDate` when in-sample or out-of-sample backtesting is requested. After that, the enxt start date would be the previous start date plus the date offset of the rebalancing period (e.g. 1 months, 2 weeks etc.). These dates are for indexing purposes, especially for CPO API to store and manage data files for each rebalancing period. The back-testing experiment will stop when the start date of the rebalancing period is larger than the `trainingEndDate`. 

The start date of a rebalancing period can be any calendar day and may not be an effective rebalancing (market) day. To determine the actual rebalancing date of the rebalancing period, CPO requires the knowledge on when the rebalancing should take place, i.e. the `rebalane_on` parameter. If `rebalance_on` is set to 'first', then CPO will use the first marketday within the rebalancing period; otherwise if `rebalance_on` is set to 'last', then the rebalancing date will be taken from the last market day of the PREVIOUS rebalancing period.

### Backtesting

Backtesting can be further divided into in-sample and out-of-sample backtestings. CPO uses the in-sample period to tune hyper-parameters of the prediction system, including selection of candidate strategies, type of prediction model to be used, and aggregation function that turns predictions into final recommendations. The tuned hyper-parameters will be saved, and user can use out-of-sample period to verify if CPO continuous to add values. Both in-sample and out-of-sample backtesting jobs require some additional parameters then those defined the portfolio.

#### In-sample backtesting

In-sample backtesting is used to determine the hyper-parameters of the CPO system. The parameters required for in-sample backtesting include:
- `trainingStartDate`, mandatory, DateTime, the start date of the first rebalancing period to be included in the experiment. 
- `trainingEndDate`, mandatory, DateTime, the experiment terminates when the start of the period exceed the `trainingEndDate`.
- `samplingProportion`, mandatory, float between 0 and 1, the fraction of base strategies to be kept. This parameter is usually set to 0.3 or 0.4.
- `debug`, optional, will output more information in the backend when set to `debug`, and will not affect the performance or prediction.

The following code submit an in-sample backtesing job between Jan and Dec 2023 for the demo portfolio defined earlier.

In [11]:
var inSampleResult = client.RunInSampleBacktest(
    portfolioParameters,
    trainingStartDate: new DateTime(2019, 01, 01),
    trainingEndDate: new DateTime(2019, 12, 31),
    samplingProportion: 0.3,
    debug: "debug");
inSampleResult

Id,317f9752-1f01-4264-a321-7c5f03ac8717
Message,job submitted for cpo in-sample backtesting.


#### Time cost of in-sample backtesting
It can take 20 - 30 min for each rebalancing period within the in-sample period, depending on no. of components in the portfolio, size of training data, and the sampling proportion. The time cost can add up quickly when the test includes many rebalancing period, eithor due to more frequently rebalancing (e.g. weekly) or longer testing period. User can use the `get-cpo-job-status` function to check current status. CPO API will return the performance metrics of the tuned model after the job is finished, and the corresponding allocations will be sent to user's email. These weights can be loaded using a separate function as well, see later section.

It is also possible that, due to the long-running nature of the back-testing jobs (both in-sample and out-of-sample), there could be connection errors that create a breaking point. In that case, **simply re-submit the training request without chaning any parameters.** There is breaking-point handling modules in CPO API that record the progress and will continue the job from the most recent breaking point. This is true for both in-sample and out-of-sample backtesting.

The following code check the progress of the back-testing submited, and output the performance if job is finished.

In [12]:
// Check status
var job = client.GetJobForId(inSampleResult.Id);
job

{"cpo_job_id":"317f9752-1f01-4264-a321-7c5f03ac8717","cpo_job_status":"PENDING","cpo_result":"{\"return\":0.0,\"risk\":0.0,\"sharpe\":0.0,\"CAGR\":0.0,\"UI\":0.0,\"UPI\":0.0,\"MaxDD\":0.0}","progress":"{\"step\":2,\"progress\":\"Training models\"}"} Id 317f9752-1f01-4264-a321-7c5f03ac8717 Status PENDING Performance {"return":0.0,"risk":0.0,"sharpe":0.0,"CAGR":0.0,"UI":0.0,"UPI":0.0,"MaxDD":0.0} Return 0 Risk 0 SharpeRatio 0 CAGR 0 UI 0 UPI 0 MaximumDrawdown 0 Progress QuantConnect.PredictNowNET.Models.Progress Step 2 Message Training models

#### Out of Sample Backteting

Out of sample backtesting simply applies the hyper-parameters tuned during in-sample periods and test if they continue to add value over a different time period. That means, out-of-sample test can only be run after in-sample tuning is finished. The required input parameters are `trainingStartDate` and `trainingEndDate`, with the same definition and format as in-sample backtesting.

It is important to keep the `trainingStartDate` parameters have the same format for in-sample and out-of-sample tests. For this example we are working on a portfolio that takes monthly rebalance on the first market day of the month, so we will keep `trainingStartDate` to the 1st of the month in OOS. Similarly, if in-sample test starts on a Monday for a weekly rebalanced portfolio, the OOS should start on the next Monday as well. 

OOS test generally runs faster than in-sample because there are fewer models to be run, but may still take 15 min for each rebalancing period.

The following code submits an out-of-sample job for the demo portfolio between Jan and Jun 2022. The prediction will be made using configurations determined during in-sample testing.

In [13]:
var outOfSampleResult = client.RunOutOfSampleBacktest(
    portfolioParameters,
    trainingStartDate: new DateTime(2019, 01, 01),
    trainingEndDate: new DateTime(2019, 12, 31),
    debug: "debug");
outOfSampleResult

Id,bc9b3524-026f-43fc-8489-7ad72f330465
Message,job submitted for cpo back-testing.


In [14]:
// Check status
var job = client.GetJobForId(outOfSampleResult.Id);
job

{"cpo_job_id":"bc9b3524-026f-43fc-8489-7ad72f330465","cpo_job_status":"PENDING","cpo_result":"{\"return\":0.0,\"risk\":0.0,\"sharpe\":0.0,\"CAGR\":0.0,\"UI\":0.0,\"UPI\":0.0,\"MaxDD\":0.0}","progress":"{\"step\":1,\"progress\":\"Training models\"}"} Id bc9b3524-026f-43fc-8489-7ad72f330465 Status PENDING Performance {"return":0.0,"risk":0.0,"sharpe":0.0,"CAGR":0.0,"UI":0.0,"UPI":0.0,"MaxDD":0.0} Return 0 Risk 0 SharpeRatio 0 CAGR 0 UI 0 UPI 0 MaximumDrawdown 0 Progress QuantConnect.PredictNowNET.Models.Progress Step 1 Message Training models

#### Load Backtesting Results

The predictions generated during in-sample and out-of-sample backtesting experiments are stored on CPO API. Users can request these results for in-sample, out-of-sample, or combined. The key parameters are, again, `trainingStartDate` and `trainingEndDate`.

The following requests the CPO allocations and its performance for the entire back-testing period (in-sample + out-of-sample).

In [15]:
var weightsByDate = client.GetBacktestWeights(
    portfolioParameters, 
    trainingStartDate: new DateTime(2019, 01, 01), 
    trainingEndDate: new DateTime(2019, 12, 31),
    debug: "debug");
weightsByDate

key value 2019-01-02 00:00:00Z key value SPY 0.0539468147 QQQ 0.0346698355 VNQ 0.0331834347 REM 0.1002935535 IEF 0.2798133149 TLT 0.1904534153 2019-02-01 00:00:00Z key value SPY 0.0446511781 QQQ 0.039910024 VNQ 0.0463260017 REM 0.0524557947 IEF 0.1055622018 TLT 0.0640442022 2019-03-01 00:00:00Z key value SPY 0.07094082 QQQ 0.0683798591 VNQ 0.0799032846 REM 0.1146294247 IEF 0.2425103942 TLT 0.110000258 2019-04-01 00:00:00Z key value SPY 0.1144147092 QQQ 0.0295901664 VNQ 0.0388845672 REM 0.0624777368 IEF 0.2810866834 TLT 0.0631862648 2019-05-01 00:00:00Z key value SPY 0.0220024309 QQQ 0.085172843 VNQ 0 REM 0.0464939378 IEF 0.1739488572 TLT 0.0202796455 2019-06-03 00:00:00Z key value SPY 0.1345840708 QQQ 0.0807885174 VNQ 0.0556767002 REM 0.0696000078 IEF 0.4032896738 TLT 0.2560610299 2019-07-01 00:00:00Z key value SPY 0.1044118656 QQQ 0.0308209834 VNQ 0.0308209834 REM 0.0352558398 IEF 0.2014542724 TLT 0.1234285339 2019-08-01 00:00:00Z key value SPY 0.080825333 QQQ 0.0217590672 VNQ 2.201030366E-20 REM 0.150314591 IEF 0.1864209511 TLT 0.0161788184 2019-09-03 00:00:00Z key value SPY 0.0556821061 QQQ 0.053568495 VNQ 0.0733993034 REM 0.0665357219 IEF 0.1423674909 TLT 0.1365604066 2019-10-01 00:00:00Z key value SPY 0.1303359727 QQQ 0.0385461294 VNQ 0.1201818964 REM 0.1446408029 IEF 0.2664095959 TLT 0.1489831939 2019-11-01 00:00:00Z key value SPY 0.0401602358 QQQ 0.0292741935 VNQ 0.0110003924 REM 0.1232162198 IEF 0.2080889046 TLT 0.0265702144 2019-12-02 00:00:00Z key value SPY 0.048315907 QQQ 0.1021682334 VNQ 0.0269679034 REM 0.1319869795 IEF 0.166060911 TLT 0.0408145264

In [16]:
var performance = client.GetBacktestPerformance(
    portfolioParameters, 
    trainingStartDate: new DateTime(2019, 01, 01), 
    trainingEndDate: new DateTime(2019, 12, 31),
    debug: "debug");
performance

Return,0.09808653639533395
Risk,0.030395491877785306
SharpeRatio,3.2270093469690146
CAGR,0.10252805795293685
UI,0.30773793331371724
UPI,31.873398036809977
MaximumDrawdown,0.016158762466234027


### Live Prediction

Live prediction means to use the tuned hyper-parameter to make prediction for an incoming rebalancing period. Live prediction requires the knowledge of a) the target rebalancing date, and b) the prediction horizon, i.e. how many market days are there in the incoming rebalancing period. User can provide the informaion using the following parameters:

- `rebalanceDate`, mandatory, DateTime, the target rebalance date.
- `nextRebalanceDate`, optional, DateTime, the next rebalance date after current target date. For example, for a weekly-rebalanced portfolio, if the `rebalanceDate` is set to '2023-10-02', the `nextRebalanceDate` would be Monday '2023-10-09'. If `nextRebalanceDate` is passed, CPO will use US market calendar to determine how many market days are there in the target rebalancing period.
- `marketDays`, optional, int, the number of market days in the incoming rebalancing period. For a weekly rebalanced portfolio, `marketDays` is usually 5 unless there is a holiday. This parameter overrides `nextRebalanceDate`.

If neithor `nextRebalanceDate` nor `marketDays` parameters are passed, CPO will infer the number of market days from rebalancing frequency by assuming 5 market days a week and 21 market days a month.

The following code submit a live prediction training request, check the progress, then load the predicted allocation.

In [17]:
var livePredictionResult = client.RunLivePrediction(
    portfolioParameters,
    rebalanceDate: new DateTime(2022, 07, 01),
    nextRebalanceDate: new DateTime(2022, 08, 01),
    debug: "debug");
livePredictionResult

Id,46db3a3b-06f1-45c0-a68e-5bdde55e5895
Message,job submitted for cpo live prediction.


In [18]:
// Check status
var job = client.GetJobForId(livePredictionResult.Id);
job

{"cpo_job_id":"46db3a3b-06f1-45c0-a68e-5bdde55e5895","cpo_job_status":"PENDING","cpo_result":"{\"return\":0.0,\"risk\":0.0,\"sharpe\":0.0,\"CAGR\":0.0,\"UI\":0.0,\"UPI\":0.0,\"MaxDD\":0.0}","progress":"{\"step\":1,\"progress\":\"Training models\"}"} Id 46db3a3b-06f1-45c0-a68e-5bdde55e5895 Status PENDING Performance {"return":0.0,"risk":0.0,"sharpe":0.0,"CAGR":0.0,"UI":0.0,"UPI":0.0,"MaxDD":0.0} Return 0 Risk 0 SharpeRatio 0 CAGR 0 UI 0 UPI 0 MaximumDrawdown 0 Progress QuantConnect.PredictNowNET.Models.Progress Step 1 Message Training models

After the job prediction is made, user can also load the result without redo the training with the following code. The key parameter is `rebalanceDate`.

In [19]:
var weightsByDate = client.GetLivePredictionWeights(
    portfolioParameters, 
    rebalanceDate: new DateTime(2022, 07, 01),
    debug: "debug");
weightsByDate

key value 2022-07-01 00:00:00Z key value SPY 0 QQQ 0 VNQ 0 REM 0 IEF 0 TLT 0